## Serving a Bokeh application in Voila

In [ ]:
import yaml
import os

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider
from bokeh.plotting import figure
from bokeh.themes import Theme
from bokeh.io import show, output_notebook

from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature

output_notebook()

Serving a Bokeh app in Voila is the same as serving it in JupyterLab, see https://docs.bokeh.org/en/latest/docs/user_guide/output/jupyter.html#jupyterlab

In [ ]:
def bkapp(doc):
    df = sea_surface_temperature.copy()
    source = ColumnDataSource(data=df)

    plot = figure(x_axis_type='datetime', y_range=(0, 25),
                  y_axis_label='Temperature (Celsius)',
                  title="Sea Surface Temperature at 43.18, -70.43")
    plot.line('time', 'temperature', source=source)

    def callback(attr, old, new):
        if new == 0:
            data = df
        else:
            data = df.rolling('{0}D'.format(new)).mean()
        source.data = ColumnDataSource.from_df(data)

    slider = Slider(start=0, end=30, value=0, step=1, title="Smoothing by N Days")
    slider.on_change('value', callback)

    doc.add_root(column(slider, plot))

    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
                height: 500
                width: 800
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: white
    """, Loader=yaml.FullLoader))

**Note**: You need to specify the Voila URL to Bokeh, by passing it to the `show` method:

```python
import os
from urllib.parse import urlparse

url = urlparse(os.environ.get("VOILA_REQUEST_URL"))

show(bkapp, notebook_url=f"{url.scheme}://{url.netloc}")
```

In [ ]:
import os
from urllib.parse import urlparse

url = urlparse(os.environ.get("VOILA_REQUEST_URL"))

show(bkapp, notebook_url=f"{url.scheme}://{url.netloc}")